In [1]:
#Importing libaries
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os
import tensorflow as tf
from keras.models import Sequential, Model
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten, Input, concatenate
import keras

from sklearn.model_selection import train_test_split
from keras.optimizers import Adam, RMSprop

from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix, classification_report
import sklearn

In [2]:
#This is my data
df = pd.read_csv("jupyter3.csv")
df = df.drop(5)

In [3]:
df.iloc[3000:3003, :]

,DEMENTED,BILLS,TAXES,SHOPPING,GAMES,STOVE,MEALPREP,PAYATTN,REMDATES,TRAVEL,DECSUB,DECIN,FILE
3001,1,2,2,8,8,2,8,1,2,3,1,1,C:\\Users\\kaiso\\Andi sf\\Image data\\Non_Dem...
3002,1,0,1,0,0,0,0,0,0,0,1,1,C:\\Users\\kaiso\\Andi sf\\Image data\\Non_Dem...
3003,1,2,2,2,8,0,1,0,1,1,1,1,C:\\Users\\kaiso\\Andi sf\\Image data\\Non_Dem...


In [4]:
pd.set_option('display.max_colwidth', None)

In [5]:
image_dict = {row[-1]:row[1:-1] for row in df.to_numpy()}
for file_num in image_dict:
    img_path = file_num
len(image_dict)

3782

In [6]:

import re
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split


# Generate image data from file paths
images = []
for img_path in df["FILE"]:
    img_path = img_path
    img = tf.keras.preprocessing.image.load_img(img_path, target_size=(224, 224))
    img_arr = tf.keras.preprocessing.image.img_to_array(img)
    images.append(img_arr)
X = np.array(images)

# Create one-hot encoded labels
y = tf.keras.utils.to_categorical(df["DEMENTED"])

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create TensorFlow Datasets
train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train))
test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test))

# Define batch size and shuffle train dataset
BATCH_SIZE = 32
train_ds = train_ds.shuffle(buffer_size=len(X_train))
train_ds = train_ds.batch(BATCH_SIZE)
test_ds = test_ds.batch(BATCH_SIZE)

# Define the regex pattern to identify the keywords in the file name
positive_pattern = r"(^|\W)demented(\W|$)"

# Define a function to extract the label from the image path
def extract_label(image_path):
    # Extract the file name from the image path
    file_name = image_path.split("/")[-1]

    # Check if the file name contains the positive keyword
    if re.search(positive_pattern, file_name):
        return 1
    else:
        return 0
import numpy as np

# Define a function to preprocess the image data and extract features
def preprocess_image(img_path, num_data):
    # Load image and convert to array
    img = tf.keras.preprocessing.image.load_img(img_path, target_size=(224, 224))
    img_arr = tf.keras.preprocessing.image.img_to_array(img)
    img_arr = np.expand_dims(img_arr, axis=0)
    
    # Apply feature extraction model and flatten output
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, (3, 3), activation="relu", input_shape=(224, 224, 3)),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(64, (3, 3), activation="relu"),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(128, (3, 3), activation="relu"),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Flatten(name="flatten"),
    ])
    flatten_output = tf.keras.backend.function(model.input, model.get_layer("flatten").output)
    features = flatten_output(img_arr)
    features = np.append(image_dict[num_data], features)
    # Extract label from image path
    label = extract_label(img_path)

    return (features, label)


In [ ]:
image_data = {}
for img_path in df["FILE"]:
    num_value = img_path
    features, label = preprocess_image(img_path, num_value)  # Access image_dict using img_path as key
    image_data[img_path] = (tuple(features), label)
    
print(list(image_data.keys()))

In [ ]:
print(image_data['C:\\\\Users\\\\kaiso\\\\Andi sf\\\\Image data\\\\Non_Demented\\\\demented (6).jpg'])

In [ ]:
#Dictionry with file paths as the keys
image_dict = {row[-1]:row[1:-1] for row in df.to_numpy()}
image_dict

In [ ]:
for key, value in image_data.items():
    image_data[key] = (tuple(value[0]) + (value[1],),)

In [ ]:
# Define the maximum number of items per dataframe
max_items_per_df = 1000

# Initialize an empty list to store the DataFrames
dfs = []

# Process the dictionary in groups of max_items_per_df items
for i in range(0, len(image_data), max_items_per_df):
    # Extract a subset of the dictionary
    subset = {k: image_data[k] for k in list(image_data.keys())[i:i+max_items_per_df]}
    
    # Convert the subset to a DataFrame with index as the keys
    subset_df = pd.DataFrame.from_dict(subset, orient='index')
    
    # If the value is a tuple, split it into separate columns
    if isinstance(subset_df.iloc[0][0], tuple):
        # Modify the tuple so that the last value is first
        subset_df[0] = subset_df[0].apply(lambda x: (x[-1],) + tuple(x[:-1]))
        
        # Create a new DataFrame from the modified tuples
        subset_df = pd.DataFrame(subset_df[0].tolist(), index=subset_df.index)
        subset_df.columns = [f"col{i+1}" for i in range(subset_df.shape[1])]
        
        # The first column is the class column
        class_values = subset_df.iloc[:, 0]
        subset_df = subset_df.iloc[:, 1:]
        subset_df.insert(0, 'class', class_values)
    
    # Append the subset DataFrame to the list
    dfs.append(subset_df)


In [ ]:
the_new_and_improved_df = pd.concat(dfs, axis=0)

In [ ]:
print(image_data['C:\\\\Users\\\\kaiso\\\\Andi sf\\\\Image data\\\\Non_Demented\\\\demented (441).jpg'])

In [ ]:
the_new_and_improved_df.iloc[3000:3003, :]

In [ ]:
the_new_and_improved_df.loc[:,"class"]

In [ ]:
y = the_new_and_improved_df[the_new_and_improved_df.columns[0]]
X = the_new_and_improved_df.drop(columns=the_new_and_improved_df.columns[0])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, train_size=0.8, random_state=42, shuffle=True)

In [ ]:
from kerastuner.tuners import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters as hp
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
import pandas as pd

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, train_size=0.8, random_state=42, shuffle=True)

# Feature scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert the target variable to categorical data
num_classes = len(y.unique())
y_train_categorical = to_categorical(y_train, num_classes)
y_test_categorical = to_categorical(y_test, num_classes)

search_space = hp()
search_space.Int('num_epochs', min_value=10, max_value=50, step=5)
search_space.Int('early_stopping_patience', min_value=5, max_value=20, step=5)
# Define the search space
#search_space = {
    #'num_epochs': best_hps.Int(name='num_epochs', min_value=10, max_value=50, step=5),
    #'early_stopping_patience': best_hps.Int(name='early_stopping_patience', min_value=5, max_value=20, step=5)
#}

def build_model(hp):
    model = Sequential()
    model.add(Dense(units=hp.Int('units', min_value=32, max_value=256, step=32), activation='relu', input_shape=(X_train_scaled.shape[1],), kernel_regularizer=tf.keras.regularizers.l2(0.2)))
    model.add(Dropout(hp.Float('dropout_1', min_value=0.1, max_value=0.5, step=0.1)))
    model.add(Dense(units=hp.Int('units_1', min_value=32, max_value=128, step=32), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.2)))
    model.add(Dense(units=hp.Int('units_2', min_value=16, max_value=64, step=16), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.2)))
    model.add(Dropout(hp.Float('dropout_2', min_value=0.1, max_value=0.5, step=0.1)))
    model.add(Dense(num_classes, activation='softmax', kernel_regularizer=tf.keras.regularizers.l1(0.2)))

    # Compile the model
    optimizer = tf.keras.optimizers.Adam(lr=hp.Choice('learning_rate', values=[0.001, 0.0001]))

    model.compile(optimizer=optimizer, loss=tf.losses.BinaryCrossentropy(), metrics=['accuracy'])

    # Add early stopping callback
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=hp.Int('early_stopping_patience', min_value=5, max_value=20, step=5))

    return model



#search_space = hp.Hyperparameter()
#search_space.add('num_epochs', hp.Int(min_value=10, max_value=50, step=5))
#search_space.add('early_stopping_patience', hp.Int(min_value=5, max_value=20, step=5))
##search_space = hp.HyperParameters()
#search_space.add('num_epochs', hp.Int(min_value=10, max_value=50, step=5))
#search_space.add('early_stopping_patience', hp.Int(min_value=5, max_value=20, step=5))



tuner = RandomSearch(
    build_model,
    objective='accuracy',
    max_trials=10,
    executions_per_trial=4,
    directory='hyperparameters',
    project_name='neural_network',
    hyperparameters=search_space
)

best_trials = tuner.oracle.get_best_trials(num_trials=1)
if best_trials:
    best_hps = best_trials[0].hyperparameters
    epochs = best_hps.get('num_epochs')
    early_stopping_patience = best_hps.get('early_stopping_patience')
else:
    epochs = 40
    early_stopping_patience = 15

tuner.search(
    x=X_train_scaled,
    y=y_train_categorical,
    epochs=epochs,
    validation_data=(X_test_scaled, y_test_categorical),
    callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=early_stopping_patience)]
)
best_hps = tuner.get_best_hyperparameters(1)[0]
print("Units: {}".format(best_hps.get('units')))
print("Dropout 1: {}".format(best_hps.get('dropout_1')))
print("Units 1: {}".format(best_hps.get('units_1')))
print("Units 2: {}".format(best_hps.get('units_2')))
print("Dropout 2: {}".format(best_hps.get('dropout_2')))
print("Learning rate: {}".format(best_hps.get('learning_rate')))
print("Number of epochs: {}".format(best_hps.get('num_epochs')))
print("Early stopping patience: {}".format(best_hps.get('early_stopping_patience')))

In [ ]:
best_hps = tuner.get_best_hyperparameters(1)[0]
print("Units: {}".format(best_hps.get('units')))
print("Dropout 1: {}".format(best_hps.get('dropout_1')))
print("Units 1: {}".format(best_hps.get('units_1')))
print("Units 2: {}".format(best_hps.get('units_2')))
print("Dropout 2: {}".format(best_hps.get('dropout_2')))
print("Learning rate: {}".format(best_hps.get('learning_rate')))
print("Number of epochs: {}".format(best_hps.get('num_epochs')))
print("Early stopping patience: {}".format(best_hps.get('early_stopping_patience')))


In [ ]:
import shutil
shutil.rmtree('hyperparameters/neural_network')


In [ ]:
log_dir = 'logs'

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, train_size=0.8, random_state=42, shuffle=True)

scaler = sklearn.preprocessing.StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert the target variable to categorical data
num_classes = len(y.unique())
y_train_categorical = keras.utils.to_categorical(y_train, num_classes)
y_test_categorical = keras.utils.to_categorical(y_test, num_classes)

from keras import regularizers

# Create a Keras model with L2 regularization
model = Sequential()
model.add(Dense(160, activation='relu', input_shape=(X_train_scaled.shape[1],), kernel_regularizer=regularizers.l2(0.2)))
model.add(Dropout(0.30000000000000000004))
model.add(Dense(96, activation='relu', kernel_regularizer=regularizers.l2(0.2)))
model.add(Dense(48, activation='relu', kernel_regularizer=regularizers.l2(0.2)))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax', kernel_regularizer=regularizers.l1(0.2)))


# Compile the model
optimizer = keras.optimizers.Adam(lr=0.0001)
model.compile(optimizer=optimizer, loss=tf.losses.BinaryCrossentropy(), metrics=['accuracy'])

# Train the model on the training data
from keras import callbacks
earlystopping = callbacks.EarlyStopping(monitor="val_loss", mode="min", patience=15, restore_best_weights=True, verbose=1)
#callbacks = [earlystopping]
hist = model.fit(X_train_scaled, y_train_categorical, epochs=20, batch_size=32, validation_data=(X_test_scaled, y_test_categorical))

In [ ]:
model.save('fin.h5')

In [ ]:
hist.history

In [ ]:
fig = plt.figure()
plt.plot(hist.history['accuracy'], color="green", label='accuracy')
plt.plot(hist.history['val_accuracy'], color="blue", label="val_accuracy")
plt.plot(hist.history['loss'], color="teal", label='loss')
plt.plot(hist.history['val_loss'], color="orange", label="val_loss")
fig.suptitle('Training performance with regu (stronger)', fontsize=15)
plt.ylabel("Score")
plt.xlabel("Epochs")
plt.legend(loc= 'upper left')
plt.show()

In [ ]:
# Evaluate the model on the test data
loss, accuracy = model.evaluate(X_test_scaled, y_test_categorical)
print(f'Test accuracy: {accuracy * 100:.2f}%')

In [ ]:
# fn

In [ ]:
model_pepper = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(name='flat'),
])

In [ ]:
featureX = tf.keras.backend.function(model_pepper.input, model_pepper.get_layer('flat').output)
##################image input#####################
in_path = 'C:\\Users\\kaiso\\Andi sf\\Image data\\Non_Demented\\non_6.jpg'#<-- image data goes here
imgIN = tf.keras.preprocessing.image.load_img(in_path, target_size=(224, 224))
imgIN = tf.keras.preprocessing.image.img_to_array(imgIN)
imgIN = np.expand_dims(imgIN, axis=0)
#############numerical input######################
numIN = np.array([0,0,0,0,0,0,0,0,0,0,0])#<--numerical data goes here

result = featureX(imgIN)
result = result.flatten() 
result = np.concatenate((result, numIN))
print(result)

In [ ]:
user_df = pd.DataFrame(result).T

In [ ]:
user_df

In [ ]:
model = load_model('C:\Users\kaiso\Andi sf\fin.ipynb')
yhat = model.predict(user_df)
yhat